In [145]:
import requests, json, xmltodict
import openai

def get_cid(bvid, max_page=10):
    """
    Get the cid of a bilibili video from its bvid.
    """

    url = "https://api.bilibili.com/x/web-interface/view?bvid=" + str(bvid)
    response = requests.get(url)
    res_dict = json.loads(response.text)

    cookie = r"LIVE_BUVID=AUTO3715893391262763; balh_mode=replace; balh_server_inner=__custom__; balh_is_closed=; buvid3=6117E460-4D82-4B15-BE1A-A55C9FD10FC618537infoc; buvid4=9518E84A-AA46-DBB3-2C8F-8030D361F84D25428-022050514-ly5N23H31hRIBxdxwOmqvQ==; buvid_fp_plain=undefined; hit-dyn-v2=1; b_nut=100; CURRENT_FNVAL=4048; _uuid=3C447151-E92D-BF57-10B97-E53BB85910C10252753infoc; rpdid=|(kkuum|~uRm0J'uY~|~mY|Ru; hit-new-style-dyn=1; SESSDATA=82dff401,1686423352,82d10*c2; bili_jct=36ea9e991cf030b5126e1d1558ac4e67; DedeUserID=14708458; DedeUserID__ckMd5=1bd4979b8de02688; i-wanna-go-back=-1; b_ut=5; nostalgia_conf=-1; header_theme_version=CLOSE; fingerprint=b2847ca1783dc8f8baadf44ec9d10a20; buvid_fp=9eb5471c66f7cf12232d2dc31ff4ce47; home_feed_column=4; PVID=2; CURRENT_QUALITY=0; bp_video_offset_14708458=769898593419001900; innersign=0; b_lsid=2CF10ED4A_186B6419CC3"

    title    = res_dict['data']['title']
    subtitle = res_dict['data']['subtitle']

    headers = {
        'Cookie': cookie,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    subtitle_url = None
    if len(data['data']['subtitle']['list']) > 0:
        subtitle_url = data['data']['subtitle']['list'][0]['subtitle_url']


    num_comments = data['data']['stat']['reply']
    comments = []

    num_pages = num_comments // 20 + 1
    for n in range(1, min(num_pages, max_page) + 1):
        url = "https://api.bilibili.com/x/v2/reply?jsonp=jsonp&pn="+str(n)+"&type=1&oid="+str(bvid)+"&sort=1&nohot=1"
        req = requests.get(url)
        text = req.text
        json_text_list = json.loads(text)

        if json_text_list["data"]["replies"] is not None:
            for i in json_text_list["data"]["replies"]:
                comments.append("%s: %s"%(i["member"]["uname"],i["content"]["message"]))

    subtitle = ""
    if subtitle_url is not None:
        subtitle_response = requests.get(subtitle_url, headers=headers)
        subtitle = ",".join([json.loads(subtitle_response.text)['body'][i]['content'] for i in range(len(json.loads(subtitle_response.text)['body']))])

    return title, subtitle, comments

def get_prompt_summarize_subtitle(title, subtitle):
    """
    Get the prompt for the given cid.
    """

    prompt = "我希望你是一名专业的视频内容编辑，帮我总结视频的内容精华。请你将视频字幕文本进行总结，然后以无序列表的方式返回，不要超过5条。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt = "我希望你是一名专业的视频内容编辑，你的工作是基于视频的字幕将视频改写成一篇文章。请你将视频字幕文本进行总结，然后以无序列表的方式返回。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt += "\n视频标题：" + title
    prompt += "\n视频字幕：" + subtitle

    return prompt

def get_prompt_summarize_comments(title, comments):
    """
    Get the prompt for the given cid.
    """

    prompt  = "我希望你是一名专业的视频内容编辑，总结评论中被大家认可的观点。请你将视频评论进行总结，然后以无序列表的方式返回，不要超过5条。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt += "\n视频标题：" + title
    prompt += "\n以下是视频评论，每条评论前有其编号："

    for i, comment in enumerate(comments):
        prompt += "\n" + str(i) + ". " + comment

    return prompt

t ,s, c = get_cid("BV1v341117LF", max_page=1)


我希望你是一名专业的视频内容编辑，你的工作是基于视频的字幕将视频改写成一篇文章。请你将视频字幕文本进行总结，然后以无序列表的方式返回。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！
视频标题：到底什么是辩证法？从古希腊到黑格尔的哲学之路
视频字幕：哈喽大家好,这里是赤色幽灵指南,致力于为您讲述改造世界的学说,出于众所周知的原因呢,原先准备好的视频现在发不出来了,我只好连夜肝出一些新视频来,这期纯哲学啊,纯哲学,这也是因为往期视频里有很多朋友留言说,希望讲一讲哲学思维的底层方法论什么的,我们怎样像马老师那样思考,那么本期视频我就带你走进马老师的老师黑格尔的哲学世界,来翻译翻译究竟什么是哲学,什么是辩证法,大家也可以多多发弹幕和评论告诉我,你还想听什么,这些都有可能成为我的下一个选题,那么接下来游戏开始,dialectic这个德语词汇最早被赫林先生翻译成辩证法引进国内,事实上这个翻译常常引起我们的误解,好像他是我们认识事物,分析问题的方法,是认识论上的东西,事实上,辩证法在黑格尔那里从来都是直达本体论的,他讲的是世界自身的运动方式,在本期视频的末尾,你将完整看到为什么辩证法是认识论本体论和逻辑学的统一,本期视频除了黑格尔之外,还将涉及到泰勒斯赫拉克利特巴门尼德,亚里士多德,笛卡尔斯宾诺莎康德,费希特和谢林,大家可以把师傅别念了,打在公屏上,哲学家,黑格尔最为看重的就是历史,他认为真理就是考察概念,在历史上的各个哲学家那里场露出了何种规定性,把所有这些因素综合起来,才构成对绝对的整体认识,所以在黑格尔之前,我们必须穿越黑格尔哲学的前史,我们的旅途毫无悬念的需要从希腊开始,几乎所有讲述辩证法的文本都会把辩证法的传统至少上溯到苏格拉底,黑格尔则把赫拉克利特之诺和苏格拉底共同作为辩证法的创始人,我们知道苏格拉底酷爱论辩,他尤其喜欢在对话中让人们一步步达到真正的认识,这种知识的助产术,也就被他称为辩证法,辩证法的本意就是对话dialog,如果你看过一些以苏格拉底为主角的对话录的话,你会发现在论辩中,他会不断提出质疑和补充,使对方不断扬弃掉错误的观念,并自己推出对事物的真正知识,在苏格拉底那里,作为一种认识方法的论辩,已经不仅仅停留在主体认识事物,辩论家之间分一个对错的层面了,而是在还原事物运动的本来样貌,使知识本身在我们的对话中产生出来,作为主体的我们反倒同时成

In [146]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user", 
    "content": get_prompt_summarize_subtitle(t, s)[:2000]
    },
  ],
  temperature = 0.5,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0,
  max_tokens = 1000,
  stream = False, n = 1,
)
print(completion["choices"][0]["message"]["content"])



- 辩证法是一种直达本体论的哲学思想，而不仅仅是认识论上的东西。
- 黑格尔认为真理是考察概念在历史上的各个哲学家那里场露出的何种规定性的综合。
- 辩证法的传统至少上溯到苏格拉底，他将赫拉克利特之诺和苏格拉底共同作为辩证法的创始人。
- 在苏格拉底那里，辩证法已经不仅仅停留在主体认识事物，而是在还原事物运动的本来样貌，使知识本身在我们的对话中产生出来。
- 古希腊的自然哲学试图通过认识世界，进而认识人，探求人如何组织自己的行动和生活的哲学。
- 泰勒斯被认为是世界上第一位哲学家，他回答了世界的本源是什么，它的答案是水，或者说是水所象征的那个生成生长的生命元素。
- 赫拉克利特的关键概念是罗格斯，他认为世界就是一团永恒不灭的火，也就是作为普遍规律的理性精神。
- 巴门尼德认为万物皆动，为他不动，他自身不动，但却是一切事物的推动者。
- 从巴门尼德开始，希腊哲学开始向更高的维度转向，称之为本体论的转向。
- 在人们的语言交流中，逐渐形成了一种基本的语句结构，就是主词系词名词的结构。


In [147]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user", 
    "content": get_prompt_summarize_comments(t, c)[:1000]
    },
  ],
  temperature = 0.5,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0,
  max_tokens = 2000,
  stream = False, n = 1,
)
print(completion["choices"][0]["message"]["content"])



3. 小猪佩奇的爹: 辩证法是一种思维方式，它通过对矛盾的分析和解决，推动事物的发展和进步。本视频通过历史的角度，深入浅出地介绍了辩证法的发展历程和主要思想，对于初学者来说十分友好易懂。

4. 花花绿绿的春天: 视频中提到了黑格尔的辩证法是倒立的，需要将其倒过来才能发现内核。这个观点十分有启发性，也让我对辩证法有了更深入的理解。

5. 红烧肉加蛋: 辩证法不仅仅可以用于哲学领域，它在实际生活中也有很多应用。例如，在解决矛盾时需要找到平衡点，在发展事物时需要考虑到矛盾的存在等等。这些都是辩证法思维的具体体现。
